In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
!pip install pydrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [0]:
downloaded = drive.CreateFile({'id': '*Paste your ID*'})
downloaded.GetContentFile('image_train_test.zip')

In [0]:
!unzip image_train_test.zip

In [0]:
import os

In [1]:
os.listdir()

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

In [0]:
# import BatchNormalization
from keras.layers.normalization import BatchNormalization

In [72]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=( 34, 50, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(75, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 33, 49, 32)        416       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 24, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 15, 23, 64)        8256      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 7, 11, 64)         0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 4928)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               630912    
_________________________________________________________________
batch_normalization_11 (Batc (None, 128)               512       
__________

In [42]:
os.listdir('image_train_test')

['train', 'test']

In [73]:
WIDTH = 50
HEIGHT = 34
BATCH_SIZE = 32
TRAIN_DIR=r'image_train_test/train'
TEST_DIR=r'image_train_test/test'
# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 900 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [74]:
EPOCHS = 5
BATCH_SIZE = 10
STEPS_PER_EPOCH = 300
VALIDATION_STEPS=20
model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS)

Epoch 1/5
300/300 [==============================] - 156s 521ms/step - loss: 2.1491 - acc: 0.2600 - val_loss: 1.9657 - val_acc: 0.2880
Epoch 2/5
300/300 [==============================] - 151s 505ms/step - loss: 1.8164 - acc: 0.3404 - val_loss: 2.3211 - val_acc: 0.3040
Epoch 3/5
300/300 [==============================] - 151s 504ms/step - loss: 1.7174 - acc: 0.3618 - val_loss: 2.1537 - val_acc: 0.2240
Epoch 4/5
300/300 [==============================] - 152s 506ms/step - loss: 1.6515 - acc: 0.3846 - val_loss: 1.8950 - val_acc: 0.3680
Epoch 5/5
300/300 [==============================] - 151s 504ms/step - loss: 1.6095 - acc: 0.4038 - val_loss: 1.6877 - val_acc: 0.3660


In [0]:
from tensorflow.python.keras.preprocessing import image

In [0]:
img = image.load_img(r'image_train_test/test/rock/rock00021.png', target_size=(34,50))

In [0]:
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]

In [96]:
pred

array([0.09090052, 0.00521583, 0.04997608, 0.09830249, 0.06428668,
       0.00474146, 0.31325158, 0.00796925, 0.01440568, 0.35095045],
      dtype=float32)

0-blues 1-classical 2-country 3-disco 4-hiphop 5-jazz 6-metal 7-pop 8-reggae 9-rock

In [100]:
np.argmax(pred)

9